In [ ]:
# Google Colab에서 실행할 경우 사용

# drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 파일 경로 지정
import sys
sys.path.append('/content/drive/MyDrive/Workspace/IA_sub_2')
print(sys.path)

import os
os.chdir('/content/drive/MyDrive/Workspace/IA_sub_2')

In [1]:
# 로컬에서 실행할 경우 사용

# 파일 경로 지정
import sys, os
from IPython import get_ipython
notebook_dir = os.path.dirname(os.path.abspath(get_ipython().run_line_magic('pwd', '')))
sys.path.append(notebook_dir)

In [2]:
# 데이터 로드
import pandas as pd
from lib import data_preprocessor

# 데이터 csv를 dataFrame으로 변환
# 데이터 경로는 ../data에 있는 data라는 이름이 들어간 csv 파일로 가정
path = '../data/data_206_col.csv'
df = data_preprocessor.transform_verilog_results_to_DataFrame(path, skiprows=0)
normal_df, data_min, data_max = data_preprocessor.normalize_DataFrame(df)
normal_df.columns

[] columns are dropped.


Index(['index', 'vth0', 'cit', 'mob_u0', 'mob_ua', 'mob_ub', 'mob_ug',
       'nfactor', 'delta', 'kg',
       ...
       'c51_low', 'c52_low', 'c53_low', 'c54_low', 'c55_low', 'c56_low',
       'c57_low', 'c58_low', 'c59_low', 'c60_low'],
      dtype='object', length=206)

In [6]:
# 데이터를 80%의 학습 데이터와 20%의 테스트 데이터로 분할
train_df, test_df = normal_df[:int(len(normal_df)*0.8)], normal_df[int(len(normal_df)*0.8):]

# 데이터를 학습 데이터와 레이블로 분할하여 dataLoader로 변환
input_feature = ['i' + str(i) + '_high' for i in range(1, 61)] + ['i' + str(i) + '_low' for i in range(1, 61)] + ['c' + str(i) + '_low' for i in range(1, 61)]
output_feature = ['vth0', 'cit', 'mob_u0', 'mob_ua', 'mob_ub', 'mob_ug', 'nfactor', 'delta', 'kg', 'nlx', 'voff', 'cf', 'cgsl', 'cgso', 'cgdo', 'cgdl', 'clc', 'delvt', 'dwc', 'cjswg', 'csdesw', 'dlbg', 'dlc', 'moin']
train_df.columns
train_dl = data_preprocessor.transform_DataFrame_to_DataLoader(train_df[input_feature], train_df[output_feature], 128, True)

# 테스트 데이터도 동일하게 변환
test_dl = data_preprocessor.transform_DataFrame_to_DataLoader(test_df[input_feature], test_df[output_feature], 1, True)

In [ ]:
# ANN 모델 학습

from lib import model_training
import lib.ANN as ANN
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

ann_model = ANN.ANN(input_feature.__len__(), output_feature.__len__())
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(ann_model.parameters(), lr=1e-5)
ann_model, train_loss, val_loss = model_training.train_model(
    ann_model, 
    train_dl, 
    100,
    criterion,
    optimizer,
    test_dl,
    1,
    True
)

torch.save(ann_model.state_dict(), 'Model/ann_model_weights.pth')

# 학습 결과 시각화
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.legend()
plt.savefig('Model/ann_model_loss.png')

In [5]:
# sklearn을 이용하기 위한 데이터 변환

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# 데이터를 학습 데이터와 레이블로 분할
features = normal_df[input_feature]
labels = normal_df[output_feature]

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Random Forest 모델 학습

from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# 랜덤 포레스트 회귀 모델 생성
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)  # n_estimators는 트리 개수, 조정 가능

# 모델 학습
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = rf_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

joblib.dump(rf_model, 'Model/random_forest_model.pkl')

In [ ]:
# GBM 모델 학습

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

# GBM 모델 생성
gbm_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
multi_output_gbm_model = MultiOutputRegressor(gbm_model)

# 모델 학습
multi_output_gbm_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = multi_output_gbm_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

joblib.dump(multi_output_gbm_model, 'Model/gbm_model.pkl')

In [ ]:
# 1-D CNN 모델 학습

from lib import model_training
import torch
import torch.nn as nn
from lib.CNN_new import CNN

cnn_model = CNN(input_feature.__len__(), output_feature.__len__())
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-5)
cnn_model, train_loss, val_loss = model_training.train_model(
    cnn_model, 
    train_dl, 
    100,
    criterion,
    optimizer,
    test_dl,
    1,
    True
)

torch.save(cnn_model.state_dict(), 'Model/cnn_model_weights.pth')

# 학습 결과 시각화
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.legend()
plt.savefig('Model/cnn_model_loss.png')

In [10]:
# GPR 모델 학습

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.multioutput import MultiOutputRegressor

# GPR 모델 생성
gpr_model = GaussianProcessRegressor(kernel=RBF() + WhiteKernel(), random_state=42)
multi_output_gpr_model = MultiOutputRegressor(gpr_model)

# 모델 학습
multi_output_gpr_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = multi_output_gpr_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

joblib.dump(multi_output_gpr_model, 'Model/gpr_model.pkl')

In [ ]:
# LSTM 모델 학습

from lib import model_training
import torch
import torch.nn as nn
from lib.LSTM import LSTM
    
lstm_model = LSTM(input_feature.__len__(), output_feature.__len__())
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=1e-5)
lstm_model, train_loss, val_loss = model_training.train_model(
    lstm_model, 
    train_dl, 
    100,
    criterion,
    optimizer,
    test_dl,
    1,
    True
)

torch.save(lstm_model.state_dict(), 'Model/lstm_model_weights.pth')

# 학습 결과 시각화
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.legend()
plt.savefig('Model/lstm_model_loss.png')